In [1]:
# import
import os
from glob import glob
import pandas as pd
import duckdb

In [2]:
# read RPL
rpl_df = pd.read_excel(open("Replenishment Repot_20 Sep 2023.xlsx", "rb"), sheet_name="Replenishment UBL_UCL", header=0, index_col=None)
rpl_df = rpl_df[['Town', 'Basepack', 'Stock on hand', 'Category description', 'Classification']]
rpl_df.columns = ['town', 'basepack', 'stock_on_hand', 'category', 'classification']
rpl_df = rpl_df.applymap(lambda s: s.upper() if type(s)==str else s)

In [3]:
# read SCCF
path = os.getcwd()
sccf_df = pd.DataFrame()
for file in glob(path + "\*SEC CCF*.xlsx"):
    file_name = os.path.basename(file)
    print("Reading data from: " + file_name)
    df = pd.read_excel(open(file, "rb"), sheet_name="Sheet1", header=1, index_col=None)
    df = df[['Local Sales Region 4', 'Pack Size', 'CS', 'CS.1']]
    df.columns = ['town', 'basepack', 'ord_qty', 'inv_qty']
    sccf_df = sccf_df.append(df)
sccf_df = sccf_df.applymap(lambda s: s.upper() if type(s)==str else s)

Reading data from: SEC CCF % 01 Sep 2023_UBL.xlsx
Reading data from: SEC CCF % 02 Sep 2023_UBL.xlsx
Reading data from: SEC CCF % 03 Sep 2023_UBL.xlsx
Reading data from: SEC CCF % 04 Sep 2023_UBL.xlsx
Reading data from: SEC CCF % 05 Sep 2023_UBL.xlsx
Reading data from: SEC CCF % 06 Sep 2023_UBL.xlsx
Reading data from: SEC CCF % 07 Sep 2023_UBL.xlsx
Reading data from: SEC CCF % 08 Sep 2023_UBL.xlsx
Reading data from: SEC CCF % 09 Sep 2023_UBL.xlsx
Reading data from: SEC CCF % 10 Sep 2023_UBL.xlsx
Reading data from: SEC CCF % 11 Sep 2023_UBL.xlsx
Reading data from: SEC CCF % 12 Sep 2023_UBL.xlsx
Reading data from: SEC CCF % 13 Sep 2023_UBL.xlsx
Reading data from: SEC CCF % 14 Sep 2023_UBL.xlsx
Reading data from: SEC CCF % 15 Sep 2023_UBL.xlsx
Reading data from: SEC CCF % 16 Sep 2023_UBL.xlsx
Reading data from: SEC CCF % 17 Sep 2023_UBL.xlsx
Reading data from: SEC CCF % 18 Sep 2023_UBL.xlsx


In [4]:
# read stock-allocation
stkalloc_df = pd.read_excel(open("Town x SKU  Allocation September'23.xlsx", "rb"), sheet_name="Town x SKU x Case x TGT ", header=2, index_col=None)
stkalloc_df = stkalloc_df[['SKU NAME', 'BUSINESS GROUP', 'CATEGORY', 'TOWN x SKU TGT - TP Cr.']]
stkalloc_df.columns = ['basepack', 'BG', 'category', 'tgt_cr' ]
stkalloc_df = stkalloc_df.applymap(lambda s: s.upper() if type(s)==str else s)

In [5]:
# read national classification
ubl_cls_df = pd.read_excel(open("UBL Classification Sep.xlsx", "rb"), sheet_name="National Classification", header=2, index_col=None)
ucl_cls_df = pd.read_excel(open("UCL Classification Sep.xlsx", "rb"), sheet_name="National Classification", header=2, index_col=None)
ntnl_cls_df = duckdb.query('''select Basepack, Classification from ubl_cls_df union select Basepack, Classification from ucl_cls_df''').df()
ntnl_cls_df.columns = ['basepack', 'classification']
ntnl_cls_df = ntnl_cls_df.applymap(lambda s: s.upper() if type(s)==str else s)

In [6]:
# read town classification
ubl_cls_df = pd.read_excel(open("UBL Classification Sep.xlsx", "rb"), sheet_name="Town SKU Classification", header=0, index_col=None)
ucl_cls_df = pd.read_excel(open("UCL Classification Sep.xlsx", "rb"), sheet_name="Town SKU Classification", header=0, index_col=None)
town_cls_df = duckdb.query('''select * from ubl_cls_df union select * from ucl_cls_df''').df()
town_cls_df.columns = ['town', 'basepack_code', 'basepack', 'sale_val', 'contrib', 'classification']
town_cls_df = town_cls_df.applymap(lambda s: s.upper() if type(s)==str else s)

In [7]:
# read national RR
rr_df = pd.read_excel(open("Working - September - Raw-L6M Sales History & RR Hana File_UBL & UCL.xlsx", "rb"), sheet_name="Sheet3", header=2, index_col=None)
rr_df = rr_df[['Row Labels', 'Sum of L3M Daily RR', 'Sum of L6M Daily RR']]
rr_df.columns = ['basepack', 'RR_3_months', 'RR_6_months']
rr_df = rr_df.applymap(lambda s: s.upper() if type(s)==str else s)

In [8]:
# town RR
town_rr_df = pd.read_excel(open("Working - September - Raw-L6M Sales History & RR Hana File_UBL & UCL.xlsx", "rb"), sheet_name="Sheet1 (2)", header=2, index_col=None)
town_rr_df = town_rr_df[['Local Sales Region 4(m.d.)', 'Pack Size(m.d.)', 'L6M Daily RR', 'L3M Daily RR']]
town_rr_df.columns = ['town', 'basepack', 'RR_3_months', 'RR_6_months']
qry = '''
select town, basepack, sum(RR_3_months) RR_3_months, sum(RR_6_months) RR_6_months
from town_rr_df 
group by 1, 2
'''
town_rr_df = duckdb.query(qry).df()
town_rr_df = town_rr_df.applymap(lambda s: s.upper() if type(s)==str else s)
display(town_rr_df)

,town,basepack,RR_3_months,RR_6_months
0,ADAMJEE,CLEAR SHAMPOO C. ACTVE CARE 5ML,2.277286,1.847631
1,ADAMJEE,CLEAR SHAMPOO COMPLETE ACTVE CARE 90ML,0.053815,0.049595
2,ADAMJEE,CLEAR SHAMPOO HIJAB ANTI LIMP 350ML,0.004464,0.000000
3,ADAMJEE,CLINIC PLUS SHMP STRNG&LONG HEALTH 340ML,0.105411,0.109131
4,ADAMJEE,CLOSE-UP THPSTE FRESHNSS MINT CHILL 45G,9.031994,9.393524
...,...,...,...,...
50086,TORKI,LIFEBUOY HAND SANITIZER TOTAL 10 200ML,0.023821,0.000000
50087,ULLAPARA,LIFEBUOY LIQUID HAND SOAP LEMON FRESH 1L,0.021421,0.023804
50088,ZINJIRA,SUNSILK SHAMPOO HFS 375ML,0.366321,0.346238
50089,WARI,GLOW&LOVELY SKN CLNS BR MULTI VITMN 100G,0.008929,0.000000


In [9]:
# RR this month
days = len(glob(path + "\*SEC CCF*.xlsx"))-2
qry = '''
select town, basepack, sum(ord_qty)*1.00/''' + str(days) + ''' RR_this_month
from sccf_df 
group by 1, 2
'''
rr_thmon_df = duckdb.query(qry).df()
rr_thmon_df = rr_thmon_df.applymap(lambda s: s.upper() if type(s)==str else s)
display(rr_thmon_df)

,town,basepack,RR_this_month
0,FENI,PEPSODENT TOOTHPASTE GERMICHECK 200G,3.253437
1,FENI,PEPSODENT TOOTHPASTE KIDS ORANGE 45G,0.092000
2,FENI,PEPSODENT TP ADV SNSTV EXPRT PRFSNL 140G,0.489563
3,FENI,PEPSODENT TP ADV SNSTV EXPRT PRFSSNL 80G,0.101625
4,FENI,GLOW & LOVELY FC WSH FOAM INSTA GLOW 50G,0.608187
...,...,...,...
42537,KULAURA,DOVE SHAMPOO MUSLIM 330ML,0.007812
42538,NOWHATA,POND'S MEN FW FOAM LIGHT OIL 100G,0.002625
42539,NAGESHWARI,TRESEMME SHAMPOO KERATIN SMOOTH 340ML,0.002625
42540,GALACHIPA,SUNSILK SHAMPOO VOLUME 195ML,0.007812


In [10]:
# BC
qry = '''
select BG, category, basepack, sum(tgt_cr)*1.00/(select sum(tgt_cr) from stkalloc_df) basepack_bc 
from stkalloc_df
group by 1, 2, 3
'''
bc_df = duckdb.query(qry).df()
bc_df = bc_df.applymap(lambda s: s.upper() if type(s)==str else s)
display(bc_df)

,BG,category,basepack,basepack_bc
0,HOME CARE,FABRIC CLEANING,RIN NM STD LIQUID 400ML,0.000316
1,HOME CARE,FABRIC CLEANING,RIN NM STD LIQUID 800ML,0.000359
2,HOME CARE,FABRIC CLEANING,RIN NM STD POWDER POWER WHITE 1000G,0.018567
3,HOME CARE,FABRIC CLEANING,RIN NM STD POWDER POWER WHITE 200G,0.009244
4,HOME CARE,FABRIC CLEANING,RIN NM STD POWDER POWER WHITE 2KG,0.001059
...,...,...,...,...
264,BEAUTY & WELLBEING,SKIN CARE,POND'S CREAM OIL CNTRL VAN CRM 50G,0.000162
265,PERSONAL CARE,SKIN CLEANSING,DOVE BAR PINK 100G,0.001489
266,PERSONAL CARE,SKIN CLEANSING,LIFEBUOY SKIN CLEANSING BAR CARE 2X100G,0.002421
267,PERSONAL CARE,SKIN CLEANSING,LIFEBUOY SKIN CLNG BAR LEMON FRESH 150G,0.000818


In [11]:
# national data
qry = '''
select BG, category, classification, basepack_bc, basepack, attr, val
from
    (select basepack, 'stock_on_hand' attr, sum(stock_on_hand) val
    from rpl_df 
    group by 1, 2
    
    union all
    
    select basepack, 'DOH_3_months' attr, stock_on_hand*1.00/RR_3_months val
    from 
        (select basepack, sum(stock_on_hand) stock_on_hand
        from rpl_df 
        group by 1
        ) tbl1
        inner join 
        (select basepack, RR_3_months
        from rr_df
        ) tbl2 using(basepack)
        
    union all
    
    select basepack, 'DOH_6_months' attr, stock_on_hand*1.00/RR_6_months val
    from 
        (select basepack, sum(stock_on_hand) stock_on_hand
        from rpl_df 
        group by 1
        ) tbl1
        inner join 
        (select basepack, RR_6_months
        from rr_df
        ) tbl2 using(basepack)
        
    union all
    
    select basepack, 'DOH_this_month' attr, stock_on_hand*1.00/RR_this_month val
    from 
        (select basepack, sum(stock_on_hand) stock_on_hand
        from rpl_df 
        group by 1
        ) tbl1
        inner join 
        (select basepack, sum(RR_this_month) RR_this_month
        from rr_thmon_df
        group by 1
        ) tbl2 using(basepack)
    
    union all
    
    select distinct basepack, 'DOH_target' attr, 18 val
    from rpl_df
    ) tbl1
    
    left join
    
    (select BG, category, basepack, basepack_bc from bc_df) tbl2 using(basepack)
    
    left join 
    
    (select basepack, classification from ntnl_cls_df) tbl3 using(basepack)
'''
ntnl_df = duckdb.query(qry).df() #.fillna('')

# pivot
piv_df = pd.pivot_table(
    ntnl_df, 
    values='val', 
    index=['BG', 'category', 'classification', 'basepack_bc', 'basepack'], 
    columns=['attr'], 
    aggfunc='sum'
    ).reset_index()
display(piv_df)

attr,BG,category,classification,basepack_bc,basepack,DOH_3_months,DOH_6_months,DOH_target,DOH_this_month,stock_on_hand
0,BEAUTY & WELLBEING,HAIR CARE,A,0.042645,SUNSILK SHAMPOO BLACK 6ML,7.691462,7.275869,18.0,6.468825,54919.0
1,BEAUTY & WELLBEING,HAIR CARE,B,0.013024,CLEAR MALE SHAMPOO CSM 5ML,9.962964,7.734319,18.0,5.316438,6468.0
2,BEAUTY & WELLBEING,HAIR CARE,C,0.002517,DOVE SHAMPOO HAIR FALL RESCUE 170ML,10.626494,10.882412,18.0,16.111427,1334.0
3,BEAUTY & WELLBEING,HAIR CARE,C,0.003117,CLEAR SHAMPOO C. ACTVE CARE 5ML,11.473847,9.097855,18.0,4.348993,3128.0
4,BEAUTY & WELLBEING,HAIR CARE,C,0.004060,SUNSILK SHAMPOO BLACK 375ML,13.151291,12.276873,18.0,13.904308,1398.0
...,...,...,...,...,...,...,...,...,...,...
230,PERSONAL CARE,SKIN CLEANSING,D,0.002119,LIFEBUOY SKIN CLEANSING BAR TOTAL BB 30G,7.665431,8.250614,18.0,6.088798,2990.0
231,PERSONAL CARE,SKIN CLEANSING,D,0.002121,LIFEBUOY SKIN CLNG BAR LEMON FRESH 30G,6.154675,7.367536,18.0,4.656832,2445.0
232,PERSONAL CARE,SKIN CLEANSING,D,0.002415,LIFEBUOY SKN CLNG BAR LEMON FRESH 2X100G,13.961707,13.961707,18.0,12.641069,1875.0
233,PERSONAL CARE,SKIN CLEANSING,D,0.002421,LIFEBUOY SKIN CLEANSING BAR CARE 2X100G,15.293810,16.330927,18.0,8.829547,1939.0


In [12]:
# town data
qry = '''
select BG, category, town, classification, basepack_bc, basepack, attr, val
from
    (select town, basepack, 'stock_on_hand' attr, sum(stock_on_hand) val
    from rpl_df 
    group by 1, 2, 3
    
    union all
    
    select town, basepack, 'DOH_3_months' attr, stock_on_hand*1.00/RR_3_months val
    from 
        (select town, basepack, sum(stock_on_hand) stock_on_hand
        from rpl_df 
        group by 1, 2
        ) tbl1
        inner join 
        (select town, basepack, RR_3_months
        from town_rr_df
        ) tbl2 using(town, basepack)
        
    union all
    
    select town, basepack, 'DOH_6_months' attr, stock_on_hand*1.00/RR_6_months val
    from 
        (select town, basepack, sum(stock_on_hand) stock_on_hand
        from rpl_df 
        group by 1, 2
        ) tbl1
        inner join 
        (select town, basepack, RR_6_months
        from town_rr_df
        ) tbl2 using(town, basepack)
        
    union all
    
    select town, basepack, 'DOH_this_month' attr, stock_on_hand*1.00/RR_this_month val
    from 
        (select town, basepack, sum(stock_on_hand) stock_on_hand
        from rpl_df 
        group by 1, 2
        ) tbl1
        inner join 
        (select town, basepack, RR_this_month
        from rr_thmon_df
        ) tbl2 using(town, basepack)
    
    union all
    
    select town, basepack, 'DOH_target' attr, 18 val
    from rpl_df
    ) tbl1
    
    left join
    
    (select BG, category, basepack, basepack_bc from bc_df) tbl2 using(basepack)
    
    left join 
    
    (select town, basepack, classification from town_cls_df) tbl3 using(town, basepack)
'''
town_df = duckdb.query(qry).df() #.fillna('')

# # pivot
# piv2_df = pd.pivot_table(
#     town_df, 
#     values='val', 
#     index=['BG', 'category', 'classification', 'basepack_bc', 'basepack'], 
#     columns=['town', 'attr'], 
#     aggfunc='sum'
#     ).reset_index()
# display(piv2_df)

# pivot
piv2_df = pd.pivot_table(
    town_df, 
    values='val', 
    index=['BG', 'category', 'classification', 'basepack_bc', 'basepack', 'town'], 
    columns=['attr'], 
    aggfunc='sum'
    ).reset_index()
display(piv2_df)

attr,BG,category,classification,basepack_bc,basepack,town,DOH_3_months,DOH_6_months,DOH_target,DOH_this_month,stock_on_hand
0,BEAUTY & WELLBEING,HAIR CARE,A,0.000824,TRESEMME SHAMPOO KERATIN SMOOTH 580ML,MODERN TRADE,1.092749,1.101371,18.0,0.971339,9.0
1,BEAUTY & WELLBEING,HAIR CARE,A,0.000824,TRESEMME SHAMPOO KERATIN SMOOTH 580ML,SMT & SHOPPING COMPL,1.821516,2.339582,18.0,2.330532,13.0
2,BEAUTY & WELLBEING,HAIR CARE,A,0.004060,SUNSILK SHAMPOO BLACK 375ML,MODERN TRADE,0.660832,0.591584,18.0,0.394896,5.0
3,BEAUTY & WELLBEING,HAIR CARE,A,0.004060,SUNSILK SHAMPOO BLACK 375ML,SMT & SHOPPING COMPL,3.245217,3.785129,18.0,2.237606,18.0
4,BEAUTY & WELLBEING,HAIR CARE,A,0.005361,CLINIC PLUS SHMP STRNG & LONG HEALTH 3ML,DHUPCHACHIYA,5.675204,5.907004,18.0,1.588530,184.0
...,...,...,...,...,...,...,...,...,...,...,...
34685,PERSONAL CARE,SKIN CLEANSING,D,0.001217,LIFEBUOY LIQUID HAND SOAP CARE 200ML,TORKI,13.507538,19.478261,18.0,9.034444,2.0
34686,PERSONAL CARE,SKIN CLEANSING,D,0.001217,LIFEBUOY LIQUID HAND SOAP CARE 200ML,ZAKIGANJ,34.757422,43.829898,18.0,76.923077,2.0
34687,PERSONAL CARE,SKIN CLEANSING,D,0.001217,LIFEBUOY LIQUID HAND SOAP CARE 200ML,PANCHAGARH,9.739130,17.456359,18.0,NaN,1.0
34688,PERSONAL CARE,SKIN CLEANSING,D,0.001217,LIFEBUOY LIQUID HAND SOAP CARE 200ML,BEANI BAZAR,20.624252,19.717153,18.0,29.533918,4.0


In [13]:
# store
with pd.ExcelWriter("C:/Users/Shithi.Maitra/Downloads/operational_file.xlsx") as writer:
    piv_df.to_excel(writer, sheet_name="National", index=False)
    # piv2_df.to_excel(writer, sheet_name="Town", index=True)
    piv2_df.to_excel(writer, sheet_name="Town", index=False)

In [14]:
# # sanity check
# qry = '''
# select town, basepack, count(*) entry
# from rpl_df
# group by 1, 2
# order by 3 desc
# '''
# df = duckdb.query(qry).df()
# display(df)